In [1]:
import os
import copy
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import itertools

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential

import hls4ml
from hls4ml.converters.keras_to_hls import parse_default_keras_layer
from hls4ml.model.attributes import ConfigurableAttribute, TypeAttribute
from hls4ml.model.types import FixedPrecisionType, RoundingMode, SaturationMode
from hls4ml.model.attributes import Attribute

import h5py

import qkeras as qk
from qkeras.estimate import print_qstats
from qkeras.utils import model_quantize
from qkeras.utils import quantized_model_dump
from qkeras import QActivation, QDense, QConv2DBatchnorm

# Source the Vivado path
os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

np.random.seed(0)
tf.random.set_seed(0)


2025-07-01 13:31:34.561576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-01 13:31:34.800110: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-01 13:31:34.810124: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-07-01 13:31:34.810166: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

/home/aelabd/RHEED/hls4ml/hls4ml/converters/__init__.py:29: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


Helper functions

In [2]:
def ap_fixed_bin_to_float(bin_str: str, X: int, Y: int) -> np.float32:
    """
    Converts a binary string in two's complement ap_fixed<X,Y> format to a float.
    
    Parameters:
        bin_str (str): Binary string of length X.
        X (int): Total number of bits.
        Y (int): Number of integer bits (including sign bit).
        
    Returns:
        np.float32: Floating point value.
    """
    if len(bin_str) != X:
        raise ValueError(f"Binary string length ({len(bin_str)}) must match X ({X})")
    
    # Interpret binary string as signed integer
    int_val = int(bin_str, 2)
    if bin_str[0] == '1':  # Negative number (two's complement)
        int_val -= (1 << X)
    
    # Scale back by number of fractional bits
    frac_bits = X - Y
    value = int_val / (2 ** frac_bits)
    return np.float32(value)

print(ap_fixed_bin_to_float('00110100', 8, 4))  # 3.25
print(ap_fixed_bin_to_float('11100000', 8, 4))  # -4

3.25
-2.0


# Load QKeras benchmarks

In [4]:
IN_ROWS = 100
IN_COLS = 160
OUT_ROWS = 48
OUT_COLS = 48
NUM_CROPS = 1
data_dir = f"tb_data_Mono8/{IN_ROWS}x{IN_COLS}_to_{OUT_ROWS}x{OUT_COLS}x{NUM_CROPS}"

CROP_X0 = [0, 13, 112]
CROP_Y0 = [0, 1, 52]
qkeras_out = {}
for y0 in CROP_Y0:
    print(f"\ny0: {y0}")
    qkeras_out[f"y1_{y0}"] = {}
    for x0 in CROP_X0:
        if y0==0 and x0==13:
            continue
        print(f"  x0: {x0}")
        crop_data = []
        fpath = os.path.join(data_dir, f"Y1_{y0}/X1_{x0}/QKeras_pred_ap_fixed_22_11.txt")
        with open(fpath, "r") as f:
            for line in f.readlines():
                line_float = ap_fixed_bin_to_float(line.strip(), 22, 11)
                qkeras_out[f"y1_{y0}"][f"x1_{x0}"] = line_float


y0: 0
  x0: 0
  x0: 112

y0: 1
  x0: 0
  x0: 13
  x0: 112

y0: 52
  x0: 0
  x0: 13
  x0: 112


# Load RTL CNN benchmarks

In [5]:
rtl_cnn_out = {}
for y0 in CROP_Y0:
    print(f"\ny0: {y0}")
    rtl_cnn_out[f"y1_{y0}"] = {}
    for x0 in CROP_X0:
        if y0==0 and x0==13:
            continue
        print(f"  x0: {x0}")
        crop_data = []
        fpath = os.path.join(data_dir, f"Y1_{y0}/X1_{x0}/CNN_out_benchmark_ap_fixed_22_11.txt")
        with open(fpath, "r") as f:
            for line in f.readlines():
                line_float = ap_fixed_bin_to_float(line.strip(), 22, 11)
                rtl_cnn_out[f"y1_{y0}"][f"x1_{x0}"] = line_float


y0: 0
  x0: 0
  x0: 112

y0: 1
  x0: 0
  x0: 13
  x0: 112

y0: 52
  x0: 0
  x0: 13
  x0: 112


In [6]:
full_pipeline_out = {}
for y0 in CROP_Y0:
    print(f"\ny0: {y0}")
    full_pipeline_out[f"y1_{y0}"] = {}
    for x0 in CROP_X0:
        if y0==0 and x0==13:
            continue
        print(f"  x0: {x0}")
        crop_data = []
        fpath = os.path.join(data_dir, f"Y1_{y0}/X1_{x0}/full_pipeline_out_ap_fixed_22_11.txt")
        with open(fpath, "r") as f:
            for line in f.readlines():
                line_float = ap_fixed_bin_to_float(line.strip(), 22, 11)
                full_pipeline_out[f"y1_{y0}"][f"x1_{x0}"] = line_float


y0: 0
  x0: 0
  x0: 112

y0: 1
  x0: 0
  x0: 13
  x0: 112

y0: 52
  x0: 0
  x0: 13
  x0: 112


# Create, imshow 3D map of diff

In [12]:
# QKeras CNN - RTL CNN 
print(f"\nQKeras CNN - RTL CNN pipeline")
cnn_diff = {}
for y0 in CROP_Y0:
    print(f"\ny0={y0}")
    cnn_diff[f"y1_{y0}"] = {}
    for x0 in CROP_X0:
        if y0==0 and x0==13:
            continue
        cnn_diff[f"y1_{y0}"][f"x1_{x0}"] = qkeras_out[f"y1_{y0}"][f"x1_{x0}"] - rtl_cnn_out[f"y1_{y0}"][f"x1_{x0}"]
        print(f"  x0={x0} diff: {cnn_diff[f'y1_{y0}'][f'x1_{x0}']:.6f}")


QKeras CNN - RTL CNN pipeline

y0=0
  x0=0 diff: 0.160156
  x0=112 diff: 0.182129

y0=1
  x0=0 diff: 0.082031
  x0=13 diff: 0.095215
  x0=112 diff: 0.086426

y0=52
  x0=0 diff: 0.156250
  x0=13 diff: 0.197754
  x0=112 diff: 0.156250


In [13]:
# RTL CNN - RTL full pipeline
rtl_diff = {}
for y0 in CROP_Y0:
    print(f"\ny0={y0}")
    rtl_diff[f"y1_{y0}"] = {}
    for x0 in CROP_X0:
        if y0==0 and x0==13:
            continue
        rtl_diff[f"y1_{y0}"][f"x1_{x0}"] = rtl_cnn_out[f"y1_{y0}"][f"x1_{x0}"] - full_pipeline_out[f"y1_{y0}"][f"x1_{x0}"]
        print(f"  x0={x0} diff: {rtl_diff[f'y1_{y0}'][f'x1_{x0}']:.6f}")
    
                


y0=0
  x0=0 diff: 0.008301
  x0=112 diff: -0.013672

y0=1
  x0=0 diff: 0.000000
  x0=13 diff: -0.015625
  x0=112 diff: -0.004395

y0=52
  x0=0 diff: -0.005859
  x0=13 diff: -0.086914
  x0=112 diff: -0.005859
